In [0]:
#install chromium, its driver, and selenium:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [1,284 kB]
Hit:14 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Fetched 1,536

In [0]:
#set options to be headless
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [0]:
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.prokabaddi.com/schedule-fixtures-results")
#print(wd.page_source) # results

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


## Webscraping for Pro Kabbaddi Website

In [0]:
from bs4 import BeautifulSoup
from selenium.webdriver.support.select import Select
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np

# create a csv for the match details and create a header
out_filename = "Match_Details.csv"
headers = "match_number,season,date,time,status,team_name1,team_name2,team1_score,team2_score,venue,result_text \n"

# opens file, and writes headers
f = open(out_filename, "w")
f.write(headers)

# create a csv for the match details and create a header
teamstats_filename = "Team_Stats.csv"
stat_header = "Team_Name,Season,Matches_Played,Wins,Draws,Losses, Finishing_Position,Highest Score,Biggest_Win_Margin,Total Raids,Succesful_Raids," +  \
              "Unsuccessful_raids,Empty_Raids, Success_Raid_%, No_Super_Raids, Raid_Touch_Points," + \
              "Raid_Bonus_Points,Total_Raid_Points,Total_Tackles,Successfull_Tackles," + \
               "Unsuccessful_Tackles,Successful_Tackle_%,No_Super_Tackles," + \
               "Allouts_Inflicted,Total_Allout_Points,Total_Defence_Points\n"
# opens file, and writes headers
f1 = open(teamstats_filename, "w")
f1.write(stat_header)



400

In [0]:
for k in range(1,7):
  fetch_season = "Season " + str(k)
  #Select the season by finding the select box option.
  get_div = Select(wd.find_element_by_class_name('si-selectBox.si-season-change'))
  #get_div.get_attribute("class")
  #get_div.select_by_index(5)
  get_div.select_by_visible_text(fetch_season)
  time.sleep(2)
  #print(wd.page_source) # results

  # Find the current season which is selected earlier. Also verify if the page is 
  # refreshing and active season matches with the selection we selected earlier 
  # from earlier code.
  soup = BeautifulSoup(wd.page_source, 'html') 
  selectseasonwrapper = soup.find("li", {"class": "si-season-wrapper"})
  Match_Season = selectseasonwrapper.div.select("span")[0].text
  container = soup.find("div", {"class": "mwl-partial-container"})
  container
  container.get_text()
  #Now parse through each match and get the match attributes.
  single_match_list = container.find_all("div", {"class": "si-fix-box si-single-match matCompleted "})
  for single_match_container in single_match_list:
    MatchNumber = single_match_container.get("data-gamecode")

    DateContainer = single_match_container.find("div", {"class": "si-fix-box-top-flexBox si-flexBox"})
    MatchDate = DateContainer.div.select("span")[0].text.replace(",", "")
    MatchDate
    MatchTime = DateContainer.select("div")[1].text.strip()
    MatchStatus = DateContainer.select("div")[2].span.text

    #find the match teams, location, final score
    matchdetails_container = single_match_container.find_all("div", {"class": "si-fullName"})
    teams = []
    for teamname in matchdetails_container:
      teams.append(teamname.get_text().replace("\n", ""))
    Match_Team1 = teams[0]
    Match_Team2 = teams[1]

    scoredetails_container = single_match_container.find("div", {"class": "matScore"})
    Match_Scores = scoredetails_container.select("span")[0].text.split(' - ')
    Match_Team1_Score = Match_Scores[0].strip()
    Match_Team2_Score = Match_Scores[1].strip() 

    venue_container = single_match_container.find("div", {"class": "mat-venue"})
    Match_Venue = venue_container.select("span")[0].text.strip()
    Match_Venue = Match_Venue.replace(",", "")

    result_container = single_match_container.find("div", {"class": "mat-resultStatus"})
    Match_Result = result_container.select("span")[0].text.strip()
    Match_Result

    tempstr = MatchNumber + "," + Match_Season + "," + MatchDate + "," + MatchTime + "," + MatchStatus  \
              + "," + Match_Team1 + "," + Match_Team2 + "," + Match_Team1_Score    \
              + "," + Match_Team2_Score + "," + Match_Venue + "," + Match_Result + "\n"
    f.write(tempstr)

In [0]:
f.close() #close the file
wd.close()

In [0]:
df = pd.read_csv("Match_Details.csv") 
df.head()

from google.colab import files
files.download('Match_Details.csv')

### Scape the data from the teams pages.

In [0]:
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.prokabaddi.com/stats")
#print(wd.page_source) # results

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [0]:
#loop through to get the links for all the individual team stats.
soup = BeautifulSoup(wd.page_source, 'html') 
teamlink_container = soup.find("li", {"class": "submenu teams"})
teamlink_container 
link_atags = teamlink_container.div.select("a")
link_list = []
for i in range(0,12):
  link_list.append("https://www.prokabaddi.com" + link_atags[i].get("href"))
wd.close()

In [0]:
#Now loop through each of the team profile links to get team stats
for i in range(len(link_list)):
  wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  wd.get(link_list[i])
  time.sleep(3)
  f1_team_name = link_list[i][link_list[i].rfind("/") + 1:link_list[i].find("-profile")].replace("-"," ")
  #print(wd.page_source) # results
  stats_soup = BeautifulSoup(wd.page_source, 'html')
  stats_container = stats_soup.find("div", {"class": "si-stats-container"})
  stats_container_right = stats_container.find("div", {"class": "si-section-data si-right"})
  stats_container_right_table = stats_container_right.find_all("div", {"class": "si-tbl-wraper"})
  season_header = []
  final_data = []
  no_of_seasons = 0
  season_counter = 0
  for table_wrapper in stats_container_right_table:
      data_blocks_wrapper = []
      table_data_containers = table_wrapper.find_all("div", {"class": "si-tbl-data"})
      for table_data in table_data_containers:
        data_blocks = table_data.find_all("div", {"class": "si-data-block"})
        data_points = []
        for data_blocks in data_blocks:
          if np.shape(table_wrapper.get("class"))[0] == 2:     #identify header block
            no_of_seasons += 1
            season_header.append(data_blocks.get_text().strip())
          else:
            tempstr = data_blocks.get_text().strip()
            if (tempstr.find('\n') != -1):
              tempstr = tempstr[0:tempstr.find('\n')]
            data_points.append(tempstr)
            #print(data_blocks.get_text())
        data_blocks_wrapper.append(data_points) 
      final_data.append(data_blocks_wrapper)
      #print(data_blocks_wrapper)     
  wd.close()
  for r in range(np.shape(final_data)[1]):     #Season level
    f1_writestr = f1_team_name + "," + season_header[r]
    for p in range(1, np.shape(final_data)[0]):  #Wrapper level  
      f1_writestr = f1_writestr + "," + ",".join(final_data[p][r])
    #print(f1_writestr)
    f1.write(f1_writestr + " \n")
  #break
f1.close()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [0]:
from google.colab import files
files.download('Team_Stats.csv')

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp Match_Details.csv /content/gdrive/My\ Drive/
!cp Team_Stats.csv /content/gdrive/My\ Drive/
